## Importing modules

In [65]:
import os
import numpy as np
import nltk
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import cohen_kappa_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import pandas as pd
import re

## Loading data

In [66]:

X = pd.read_csv(os.path.join('./data/', 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')


In [67]:
X.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pre processing Data

In [68]:
# we only need essay id essay_set essay domain1_score
# removing rater1_domain1   , rater2_domain1
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])
# removing NaN values

X = X.dropna(axis=1)

X.head()


,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [69]:
# loading y values to y
y = X['domain1_score']
y

0         8
1         9
2         7
3        10
4         8
         ..
12971    35
12972    32
12973    40
12974    40
12975    40
Name: domain1_score, Length: 12976, dtype: int64

Minimum and Maximum Scores for each essay set.

In [70]:
X.keys()

Index(['essay_id', 'essay_set', 'essay', 'domain1_score'], dtype='object')

In [71]:
# number of essay set
print('number of essay set : ',len(X['essay_set'].unique()))
# number of essays in each set
n_essay = X.groupby(['essay_set'], as_index=False)['domain1_score'].size()
print('\n----------total essays in each set--------\n',n_essay,'\n\ntotal : ', n_essay['size'].sum())

number of essay set :  8

----------total essays in each set--------
    essay_set  size
0          1  1783
1          2  1800
2          3  1726
3          4  1770
4          5  1805
5          6  1800
6          7  1569
7          8   723 

total :  12976


In [72]:
# max and min score of each set
max_score = X.groupby(['essay_set'], as_index=False)['domain1_score'].max()
min_score = X.groupby(['essay_set'], as_index=False)['domain1_score'].min()
print('\n----------max score of each set--------\n',max_score,'\n\n','\n----------min score of each set--------\n', min_score)


----------max score of each set--------
    essay_set  domain1_score
0          1             12
1          2              6
2          3              3
3          4              3
4          5              4
5          6              4
6          7             24
7          8             60 

 
----------min score of each set--------
    essay_set  domain1_score
0          1              2
1          2              1
2          3              0
3          4              0
4          5              0
5          6              0
6          7              2
7          8             10


In [73]:
minimum_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
maximum_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

## LSTM model

2 LSTM 1 Drop out 1 Dense

LSTM 1 300 hidden units

LSTM 2 64 hidden units

Dropout layer  0.5 dropout rate

Dense activation relu

loss = MSE and optimizer = rmsprop (0.9 default)

In [74]:

def get_model():
    
    model = Sequential()
    # 300 hidden units
    model.add(LSTM(300, dropout=0.5, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    # 64 hidden units
    model.add(LSTM(64, recurrent_dropout=0.4))
    #0.5 dropout rate
    model.add(Dropout(0.5))
    # activaion relu
    model.add(Dense(1, activation='relu'))
    #rmsprop
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    
    return model

## Essay embedding layer

In [75]:


def tokenize_essay(essay_v):
    
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    # making lower case
    words = essay_v.lower()
    words = words.split()
    len_words = len(words)
    # removing stop words
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    if(len(words)>0):
        l = len(words)
        #print(l)
    return (words)

def senteces_list(essay_v):
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    #print(raw_sentences)
    sentences = []
    for raw_sentence in raw_sentences:
        sent = tokenize_essay(raw_sentence)
        if len(sent) > 0:
            sentences.append(sent)
        else:
            warning_user = "no words in sent"
            #print(warning_user)
    return sentences

# referece from word2vec stackoverflow
def makeFeatureVec(words, model, num_features):
   # crating feature vec 
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model.wv[word])  
    # getting average of sentence vec      
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

## Preparing for Training



We will use 5-Fold Cross Validation and measure the Quadratic Weighted Kappa for each fold for final report
This is a training with single test train split

In [76]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#creating test and train essays
train_essays = X_train['essay']
test_essays = X_test['essay']

# len of train and test essay
len_train_essays = len(train_essays)
len_test_essays = len(test_essays)
#print(len_train_essays,len_test_essays)

# spliting into sentences 
sentences = []
for essay in train_essays:
            sentences += senteces_list(essay)
            


        

## Embedding layer

word2vec embedding acts as embedding layer

300 feature vectors are used

In [78]:
# creating a word2 vec model
model = Word2Vec(sentences, workers=4, vector_size=300, min_count = 40, window = 10, sample = 1e-3)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

#from word2 vec creating  embedding
clean_train_essays = []
    

for essay_v in train_essays:
    clean_train_essays.append(tokenize_essay(essay_v))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, 300)
    
clean_test_essays = []
for essay_v in test_essays:
    clean_test_essays.append(tokenize_essay( essay_v))
testDataVecs = getAvgFeatureVecs( clean_test_essays, model, 300 )
    
trainDataVecs = np.array(trainDataVecs)
testDataVecs = np.array(testDataVecs)



/var/folders/fq/fprdqqg93d3cp30hvc240yqr0000gn/T/ipykernel_44021/2316545788.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


## Training model

In [79]:
train_shape = trainDataVecs.shape
test_shape = testDataVecs.shape
#print(train_shape,test_shape)

# Reshaping train and test vectors to 3 dimensions
trainDataVecs = np.reshape(trainDataVecs, (train_shape[0], 1, train_shape[1]))
testDataVecs = np.reshape(testDataVecs, (test_shape[0], 1, test_shape[1]))
    
lstm_model = get_model()
print("model loaded")

print("Training model")
lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=150)
print("Traiing completed")



model loaded
Training model
Epoch 1/150
136/136 [==============================] - 5s 15ms/step - loss: 68.2200 - mae: 4.3279
Epoch 2/150
136/136 [==============================] - 2s 17ms/step - loss: 39.9629 - mae: 3.4785
Epoch 3/150
136/136 [==============================] - 2s 15ms/step - loss: 32.2575 - mae: 3.2785
Epoch 4/150
136/136 [==============================] - 2s 15ms/step - loss: 28.0136 - mae: 3.1316
Epoch 5/150
136/136 [==============================] - 2s 16ms/step - loss: 25.3539 - mae: 2.9663
Epoch 6/150
136/136 [==============================] - 2s 16ms/step - loss: 23.1706 - mae: 2.8037
Epoch 7/150
136/136 [==============================] - 2s 15ms/step - loss: 21.3353 - mae: 2.6566
Epoch 8/150
136/136 [==============================] - 2s 15ms/step - loss: 19.3351 - mae: 2.5174
Epoch 9/150
136/136 [==============================] - 2s 15ms/step - loss: 17.3877 - mae: 2.3756
Epoch 10/150
136/136 [==============================] - 2s 14ms/step - loss: 15.2431 - mae

## Evaluate model

In [80]:
y_pred = lstm_model.predict(testDataVecs)
    

y_pred = np.around(y_pred)
    
# Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
print("Kappa Score: {}".format(result))

Kappa Score: 0.9621362134889517


In [81]:
import warnings
warnings.filterwarnings("ignore")

#train_essays = ['Gautama Buddha, popularly known as the Buddha or Lord Buddha also known as Siddhattha Gotama or Siddhārtha Gautama or Buddha Shakyamuni, was a Śramaṇa who lived in ancient India (c. 6th to 5th century BCE or century BCE). He is regarded as the founder of the world religion of Buddhism, and revered by most Buddhist schools as a savior, the Enlightened One who rediscovered an ancient path to release clinging and craving and escape the cycle of birth and rebirth. He taught for around 45 years and built a large following, both monastic and lay. His teaching is based on his insight into the arising of duḥkha (the unsatisfactoriness of clinging to impermanent states and things) and the ending of duhkha—the state called Nibbāna or Nirvana (extinguishing of the three fires).']
train_essays = ['Covid19 taught us many things. It is possible to conduct meetings without travelling. Therefore the number of meeting have increased many times over. This has made decision making slower. ']

      
sentences = []
    
for essay in train_essays:
            sentences += senteces_list(essay)

clean_train_essays = []
    
# Generate  word vectors.
for essay_v in train_essays:
    clean_train_essays.append(tokenize_essay(essay_v))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, 300)
    
    
trainDataVecs = np.array(trainDataVecs)
train_shape = trainDataVecs.shape

#print(train_shape)
trainDataVecs = np.reshape(trainDataVecs, (train_shape[0], 1, train_shape[1]))

    
#lstm_model = get_model()
#print("here")
#lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
#lstm_model.load_weights('./model_weights/final_lstm.h5')
y_pred = lstm_model.predict(trainDataVecs)
    

    
# Round y_pred to the nearest integer.
y_pred = np.around(y_pred)
#print(y_pred)
    
final_marks = (y_pred[0][0]/60)*10
print("----your score is out of 10---------")
print(final_marks)

        

----your score is out of 10---------
1.8333333333333333
